# Live Cam Detection with Classification

## Import Packages

In [25]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to c:\users\massi\appdata\local\temp\pip-req-build-hi4d9fhs
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp37-none-any.whl size=8385 sha256=5cbfb79f71ccd352eb114b31d488313813c28b5d0e860bca06ee3d2e75fbcb5d
  Stored in directory: C:\Users\massi\AppData\Local\Temp\pip-ephem-wheel-cache-4na__5fs\wheels\36\07\46\06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


  Running command git clone -q https://github.com/rcmalli/keras-vggface.git 'C:\Users\massi\AppData\Local\Temp\pip-req-build-hi4d9fhs'


In [11]:
import numpy as np
import cv2 as cv
from cv2 import VideoCapture as cap
import matplotlib.pyplot as plt
from skimage import io
import os
import math
import time
from keras.models import load_model
from keras_vggface.utils import preprocess_input

## Load Model and define Function

In [12]:
model = load_model('weights-senet50-full2.best.hdf5')
print('Loaded model from disk')

Loaded model from disk


In [3]:
def labelling(img, model):
    #-- fase di preprocess per l'immagine (to vggface input)
    img = np.expand_dims(img, axis = 0)
    x = img.astype('float64')
    x = preprocess_input(x, version=2)
    
    #-- predict sul modello pre-trainato
    predicted = model.predict(x)
    
    #if np.max(predicted) >= .5:
    #-- acquiszione valore di accuracy
    acc = round(np.max(predicted), 3)
    
    #-- selezione per il labelling
    pred = np.argmax(predicted, axis = 1)
    if pred == 0:
        label = 'Angry'
    if pred == 1:
        label = 'Disgust'
    if pred == 2:
        label = 'Fear'
    if pred == 3:
        label = 'Happy'
    if pred == 4:
        label = 'Neutral'
    if pred == 5:
        label = 'Sad'
    if pred == 6:
        label = 'Surprise'
#     else:
#         label = 'Unknown'

    return label, acc

In [4]:
def detect(image, model, classifier,
           scale_factor = 1.1, minNeighbors = 5,
           size = (224,224)):
    '''
    This function takes in input images and after face detection apply a crop and a resize to that images.
    
    @params:
    from_directory = where to find the photos
    to_directory = where to save the photos
    classifier = face detector pre-trained (XML file)
    size = tuple
    color = RGB or BW
    init_number = first number to start naming photos
    '''
    face_cascade = cv.CascadeClassifier(classifier)
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scale_factor, minNeighbors)

    if len(faces) == 1:
        for (x, y, w, h) in faces:
            roi_color = image[y:y+h, x:x+w]
            n_img = cv.resize(roi_color, size)
            
            #-- color B, G, R
            color_face = (255,204,0) #azzurro
            color_text = (0,153,255) #arancione
            rectangle_bgr = (255,204,0) #arancione
            
            #-- setup font e size per il testo
            font = cv.FONT_HERSHEY_DUPLEX
            font_scale = 1
            
            #-- rettangolo face detection
            cv.rectangle(image, (x,y), (x+w,y+h), color_face, 3)
            #-- labelling (con model)
            label, acc = labelling(n_img, model)
            #-- preparo la stringa da stampare
            string = label + " - " + str(acc)
            #-- rettangolo per background stringa
            (text_width, text_height) = cv.getTextSize(string, font, fontScale=font_scale, thickness=1)[0]
            box_coords = ((x-2, y), (x + text_width + 4, y - text_height - 8))
            cv.rectangle(image, box_coords[0], box_coords[1], rectangle_bgr, cv.FILLED)
            #-- inserimento stringa
            cv.putText(image, string, (x,y-5), font, font_scale, color_text, 1)
            
    else:
        pass

In [5]:
def live_cam(model, classifier, resolution = (1280, 720), fps = 30, time_sleep = 2):
    '''
    This function allow to access to pc webcam in order to acquire images of the subject.
    
    @params:
    directory = where to save the captured images
    subject = name of the person who is in photo
    init_number = first number to start naming photos
    counter_limit = if desired, limit the number of photos captured
    resolution = (width, height)
    fps = Frame Per Second
    time_sleep = sleep between two photos
    numbered = Bool, necessary in order to apply counter_limit
    '''
    cam = cv.VideoCapture(0)

    cam.set(cv.CAP_PROP_FRAME_WIDTH, resolution[0])
    cam.set(cv.CAP_PROP_FRAME_HEIGHT, resolution[1])
    cam.set(cv.CAP_PROP_FPS, fps)

    cv.namedWindow("acquisition window")

    while True:
        ret, frame = cam.read()
        if not ret:
            break
        k = cv.waitKey(1)

        time.sleep(time_sleep)

        detect(frame, model, classifier=classifier, scale_factor = 1.1, minNeighbors = 6)
        
        text = 'Press ESC to exit from demo...'
        cv.putText(frame, text, (10,20), cv.FONT_HERSHEY_DUPLEX, 0.75, (255,255,255), 1)
        
        cv.imshow("acquisition window", frame)

        
        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        

    cam.release()
    cv.destroyAllWindows()

## DEMO

In [10]:
live_cam(time_sleep = 0.01, model = model, classifier = 'haarcascade_frontalface_alt2.xml')

Escape hit, closing...
